# Fine-tuning YOLOv8 on ImageNet (15 Categories)

## Objectif
Ce notebook effectue le fine-tuning du modèle YOLOv8n pré-entraîné sur un sous-ensemble de 15 catégories ImageNet pour améliorer la détection d'objets pour les 15 categories.

## Contenu
- Chargement et préparation des données ImageNet
- Configuration du modèle YOLOv8n
- Entraînement et validation
- Évaluation des performances

In [5]:
!pip install ultralytics opencv-python scikit-image pandas matplotlib


In [6]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random


In [8]:
ROOT = Path(r"C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1")
DATA_ROOT = ROOT / "DATA"
YOLO_DATASET = DATA_ROOT / "dataset_yolo"

print("ROOT:", ROOT)
print("DATA_ROOT:", DATA_ROOT)
print("YOLO_DATASET:", YOLO_DATASET)

train_imgs = list((YOLO_DATASET / "images" / "train").glob("*.*"))
val_imgs   = list((YOLO_DATASET / "images" / "val").glob("*.*"))
train_lbls = list((YOLO_DATASET / "labels" / "train").glob("*.txt"))
val_lbls   = list((YOLO_DATASET / "labels" / "val").glob("*.txt"))

print("Nb images train :", len(train_imgs))
print("Nb labels train :", len(train_lbls))
print("Nb images val   :", len(val_imgs))
print("Nb labels val   :", len(val_lbls))


ROOT: C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1
DATA_ROOT: C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1/DATA
YOLO_DATASET: C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1/DATA/dataset_yolo
Nb images train : 0
Nb labels train : 0
Nb images val   : 0
Nb labels val   : 0


In [9]:
sample_label = random.choice(train_lbls)
print("Label choisi :", sample_label)

print(sample_label.read_text())

img_name = sample_label.stem
img_path = YOLO_DATASET / "images" / "train" / f"{img_name}.JPEG"
if not img_path.exists():
    # peut-être .jpg ou .png
    for ext in [".jpg", ".JPG", ".png"]:
        p = YOLO_DATASET / "images" / "train" / f"{img_name}{ext}"
        if p.exists():
            img_path = p
            break

print("Image associée :", img_path, img_path.exists())


IndexError: Cannot choose from an empty sequence

In [ ]:
def show_yolo_bbox(img_path, label_path):
    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]
    boxes = []
    for line in label_path.read_text().strip().splitlines():
        cid, xc, yc, bw, bh = map(float, line.split())
        xc *= w; yc *= h; bw *= w; bh *= h
        x1 = int(xc - bw/2); y1 = int(yc - bh/2)
        x2 = int(xc + bw/2); y2 = int(yc + bh/2)
        boxes.append((int(cid), x1, y1, x2, y2))

    img_vis = img.copy()
    for cid, x1, y1, x2, y2 in boxes:
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img_vis, str(int(cid)), (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    plt.figure(figsize=(4,4))
    plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(img_path.name)
    plt.show()

show_yolo_bbox(img_path, sample_label)


In [ ]:
yaml_path = YOLO_DATASET / "dataset_yolo.yaml"

yaml_content = f"""
path: {YOLO_DATASET.as_posix()}
train: images/train
val: images/val

names:
  0: dog
  1: cat
  2: horse
  3: sheep
  4: bird
  5: chair
  6: bottle
  7: ball
  8: umbrella
  9: fork
  10: car
  11: motorcycle
  12:  airplane
  13: bicycle
  14: boat
"""

yaml_path.write_text(yaml_content)
print(yaml_path.read_text())


In [ ]:
base_model_path = ROOT / "yolov8n.pt"
model = YOLO(str(base_model_path))

results = model.train(
    data=str(yaml_path),
    imgsz=640,
    epochs=100,       # tu peux ajuster
    batch=16,
    device="0"     # "0" si GPU
)

print("Dossier de sortie entraînement :", results.save_dir)


In [ ]:
finetuned_dir = Path(results.save_dir)
finetuned_weights = finetuned_dir / "weights" / "final.pt"

print("Poids finetunés :", finetuned_weights, finetuned_weights.exists())

model_ft = YOLO(str(finetuned_weights))


In [ ]:
metrics = model_ft.val(data=str(yaml_path), split="val", imgsz=640, device="cpu")
metrics


In [ ]:
results_csv = finetuned_dir / "results.csv"
print(results_csv, results_csv.exists())

if results_csv.exists():
    df_metrics = pd.read_csv(results_csv)
    display(df_metrics.tail())


In [ ]:
class_names = [
    "dog","cat","horse","sheep","bird",
    "chair","bottle","ball","umbrella","fork",
    "car","motorcycle","airplane","bicycle","boat"
]

try:
    maps = list(metrics.box.maps)  # un score par classe
    df_cls = pd.DataFrame({
        "class_id": list(range(len(class_names))),
        "class_name": class_names,
        "mAP": maps
    })
    display(df_cls)
except Exception as e:
    print("Impossible de récupérer maps :", e)


In [ ]:
def visualize_predictions(model, img_paths, class_names, conf=0.4):
    for img_path in img_paths:
        img = cv2.imread(str(img_path))
        results = model(img, conf=conf, verbose=False)[0]

        img_vis = img.copy()
        h, w = img_vis.shape[:2]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            cid = int(box.cls[0])
            score = float(box.conf[0])

            if cid < 0 or cid >= len(class_names):
                continue

            label = f"{class_names[cid]} {score:.2f}"
            cv2.rectangle(img_vis, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img_vis, label, (x1, max(0, y1-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

        plt.figure(figsize=(4,4))
        plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(img_path.name)
        plt.show()

# Tester sur quelques images de validation
val_imgs = list((YOLO_DATASET / "images" / "val").glob("*.*"))
sample_val_imgs = random.sample(val_imgs, min(5, len(val_imgs)))

visualize_predictions(model_ft, sample_val_imgs, class_names, conf=0.4)


In [ ]:
test_img_path = ROOT / "test.jpg"   # remplace par une vraie image
print(test_img_path, test_img_path.exists())

if test_img_path.exists():
    visualize_predictions(model_ft, [test_img_path], class_names, conf=0.4)
